<a href="https://colab.research.google.com/github/Agathonejriii/agathonejriii/blob/main/SCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score



In [ ]:
import os

# Define the path to your project folder
project_folder = '/content/drive/MyDrive/archive.data'

# Check the contents of the main folder
os.listdir(project_folder)


# New Section

In [ ]:
import os
from PIL import Image

# Path to the "negative" and "positive" folders
negative_folder = os.path.join(project_folder, 'Negative')
positive_folder = os.path.join(project_folder, 'Positive')

# Check the contents of the "negative" folder
os.listdir(negative_folder)
os.listdir(positive_folder)


In [ ]:
# Define the paths to the "labelled" and "unlabelled" subfolders
negative_clear_folder = os.path.join(negative_folder, 'Clear')


positive_labelled_folder = os.path.join(positive_folder, 'Labelled')
positive_unlabelled_folder = os.path.join(positive_folder, 'Unlabelled')

# List the images in the labelled folder
negative_Clear_images = os.listdir(negative_clear_folder)
positive_labelled_images = os.listdir(positive_labelled_folder)
positive_unlabelled_images = os.listdir(positive_unlabelled_folder)

print("Negative Clear Images:", negative_Clear_images)
print("Positive Labelled Images:", positive_labelled_images)
print("Positive Unlabelled Images:", positive_unlabelled_images)


In [ ]:
from PIL import Image
import numpy as np

def load_image(image_path):
    """Load an image and convert it to a numpy array."""
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize to standard size (optional)
    return np.array(img)

# Example: Load the first image from the labelled negative folder
first_negative_image = os.path.join(negative_clear_folder, negative_Clear_images[0])
img_array = load_image(first_negative_image)

# Show the image
Image.open(first_negative_image).show()

print(f"Image array shape: {img_array.shape}")


In [ ]:
image_paths = []
labels = []

# Assign labels to negative and positive images
for image_name in negative_Clear_images:
    image_paths.append(os.path.join(negative_clear_folder, image_name))
    labels.append(0)  # Label 0 for negative images

for image_name in positive_labelled_images:
    image_paths.append(os.path.join(positive_labelled_folder, image_name))
    labels.append(1)  # Label 1 for positive images

# Example: Print paths and labels for first few images
for i in range(5):
    print(f"Image: {image_paths[i]}, Label: {labels[i]}")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Example: Augment a single image
img = Image.open(image_paths[0])  # Load an example image
img_array = np.array(img)
img_array = img_array.reshape((1,) + img_array.shape)  # Reshape for augmentation

# Apply the augmentation
augmented_images = datagen.flow(img_array)

# Display a few augmented images
for i in range(5):
    batch = next(augmented_images)
    augmented_img = batch[0].astype('uint8')
    Image.fromarray(augmented_img).show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define a generator for training data with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

# Create a generator for the training data
train_generator = train_datagen.flow_from_directory(
    project_folder,  # The root folder where the 'negative' and 'positive' directories are located
    target_size=(224, 224),  # Resize images to a fixed size
    batch_size=32,  # Number of images to process per batch
    class_mode='binary',  # Binary classification (0 for negative, 1 for positive)
    shuffle=True  # Shuffle images before training
)

# Check the class labels and directory structure
print(train_generator.class_indices)


In [ ]:
# Show a batch of images and their labels
x_batch, y_batch = next(train_generator)  # Fetch a single batch
print("Batch of images:", x_batch.shape)  # (batch_size, height, width, channels)
print("Labels:", y_batch)  # Corresponding labels for the batch


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to see the architecture
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_generator,  # The image generator
    epochs=10,  # Number of epochs
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of batches per epoch
)


In [ ]:
import matplotlib.pyplot as plt

# Plot training accuracy and loss
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label='loss')
plt.title('Model Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.show()
